In [2]:
import pandas as pd
import numpy as np
import sklearn 
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.pipeline import Pipeline
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
import pickle

In [26]:
recently_played=pd.read_csv('recently_played.csv')
event_data=pd.read_csv('event_data.csv')


In [27]:
recently_played.shape

(41, 12)

In [28]:
recently_played.dropna(thresh=5, axis=1)

,artist_id,artist,genre,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9
0,23fqKkggKUBHNkbKtXEls4,Kygo,'edm','pop','pop dance','tropical house',NaN,NaN,NaN,NaN,NaN
1,3NPpFNZtSTHheNBaWC82rB,X Ambassadors,'modern alternative rock','modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN
2,6TIYQ3jFPwQSRmorSezPxX,Machine Gun Kelly,'ohio hip hop',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3NPpFNZtSTHheNBaWC82rB,X Ambassadors,'modern alternative rock','modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN
4,3NPpFNZtSTHheNBaWC82rB,X Ambassadors,'modern alternative rock','modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN
5,4WN5naL3ofxrVBgFpguzKo,Rudimental,'dance pop','edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop'
6,5KKpBU5eC2tJDzf0wmlRp2,RAYE,'dance pop','pop','pop dance','post-teen pop','tropical house','uk contemporary r&b','uk pop',NaN,NaN
7,4WN5naL3ofxrVBgFpguzKo,Rudimental,'dance pop','edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop'
8,2vf4pRsEY6LpL5tKmqWb64,Elderbrook,'house','tropical house',NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4WN5naL3ofxrVBgFpguzKo,Rudimental,'dance pop','edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop'


In [5]:
event_data.head()

,clubs,djs,genres
0,40seconds Club,lilith-ge,dub techno
1,40seconds Club,upsammy,dance pop
2,40seconds Club,richii,dub techno
3,40seconds Club,samkoh,chanson
4,40seconds Club,smuseb,compositional ambient


In [22]:
query = recently_played.iloc[:30]
query =query.dropna(thresh=5, axis=1)
query

,artist_id,artist,genre,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9
0,8,10,8,'pop','pop dance','tropical house',NaN,NaN,NaN,NaN,NaN
1,13,24,13,'modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN
2,23,11,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,24,13,'modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN
4,13,24,13,'modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN
5,17,16,4,'edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop'
6,20,15,4,'pop','pop dance','post-teen pop','tropical house','uk contemporary r&b','uk pop',NaN,NaN
7,17,16,4,'edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop'
8,9,7,11,'tropical house',NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,17,16,4,'edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop'


First I need to Feature engineer the data with label encoding and onehotencoding

In [23]:
ed=event_data
rp=recently_played

In [8]:
label_encoder = preprocessing.LabelEncoder()
ed['clubs']= label_encoder.fit_transform(ed['clubs'])
ed['djs']= label_encoder.fit_transform(ed['djs'])
ed['genres']= label_encoder.fit_transform(ed['genres'])
ed


,clubs,djs,genres
0,0,316,45
1,0,549,34
2,0,457,45
3,0,474,24
4,0,498,30
...,...,...,...
566,60,161,74
567,60,191,83
568,60,468,38
569,60,99,5


In [25]:
ed=StandardScaler().fit_transform(ed)
ed

array([[-1.76250927,  0.18806885, -0.58996792],
       [-1.76250927,  1.60161861, -0.86584556],
       [-1.76250927,  1.04347879, -0.58996792],
       ...,
       [ 1.77801067,  1.1102129 , -0.76552641],
       [ 1.77801067, -1.12841311, -1.59315934],
       [ 1.77801067,  1.4681504 ,  1.59197343]])

In [9]:
rp['artist_id']=label_encoder.fit_transform(rp['artist_id'])
rp['artist']=label_encoder.fit_transform(rp['artist'])
rp['genre']=label_encoder.fit_transform(rp['genre'])
rp

,artist_id,artist,genre,genre2,genre3,genre4,genre5,genre6,genre7,genre8,genre9,genre10
0,8,10,8,'pop','pop dance','tropical house',NaN,NaN,NaN,NaN,NaN,NaN
1,13,24,13,'modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN,NaN
2,23,11,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13,24,13,'modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN,NaN
4,13,24,13,'modern rock','pop','pop rap','pop rock','rock','stomp pop',NaN,NaN,NaN
5,17,16,4,'edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop',NaN
6,20,15,4,'pop','pop dance','post-teen pop','tropical house','uk contemporary r&b','uk pop',NaN,NaN,NaN
7,17,16,4,'edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop',NaN
8,9,7,11,'tropical house',NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,17,16,4,'edm','house','pop','pop dance','tropical house','uk dance','uk funky','uk pop',NaN


In [10]:
with open('ed.pickle', 'wb') as f:
    pickle.dump(ed, f)

In [11]:
with open('ed.pickle','rb') as f:
    ed = pickle.load(f)

In [12]:
with open('rp.pickle', 'wb') as f:
    pickle.dump(rp, f)

In [13]:
with open('rp.pickle','rb') as f:
    rp = pickle.load(f)

In [14]:
ed_t=OneHotEncoder(sparse=False).fit_transform(ed)
rp_t=OneHotEncoder(sparse=False).fit_transform(rp)

In [15]:
ed_t=StandardScaler(sparse=False).fit_transform(ed_t)

TypeError: __init__() got an unexpected keyword argument 'sparse'

In [ ]:
ed_t

In [ ]:
ed_tt=pd.DataFrame(ed_t)
ed_tt

random forest classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.dummy import DummyClassifier


In [ ]:
X= ed_t[['genres']]
y= ed_t['clubs']


In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42 )

In [ ]:
model= LogisticRegression()
clf=RandomForestClassifier(max_depth=5 , random_state=42, bootstrap=True,  )

In [ ]:
model.fit(X_train, y_train)